## Introducción

Comenzaremos con una visión general de cómo funcionan los modelos de aprendizaje automático y cómo se utilizan. Esto puede parecer básico si ya has trabajado en modelado estadístico o aprendizaje automático. No te preocupes, pronto avanzaremos hacia la construcción de modelos poderosos.

En este curso, construirás modelos a medida que avances siguiendo el siguiente escenario:

Tu primo ha ganado millones de dólares especulando con bienes raíces. Te ha ofrecido asociarte en los negocios debido a tu interés en la ciencia de datos. Él aportará el dinero y tú aportarás modelos que predigan el valor de diversas casas.

Le preguntas a tu primo cómo ha predicho los valores de los bienes raíces en el pasado, y él dice que solo es intuición. Pero al hacerle más preguntas, descubres que ha identificado patrones de precios de casas que ha visto en el pasado, y utiliza esos patrones para hacer predicciones sobre nuevas casas que está considerando.

El aprendizaje automático funciona de la misma manera. Comenzaremos con un modelo llamado Árbol de Decisión. Hay modelos más sofisticados que ofrecen predicciones más precisas. Pero los árboles de decisión son fáciles de entender y son el bloque de construcción básico de algunos de los mejores modelos en ciencia de datos.

Para simplificar, comenzaremos con el árbol de decisión más simple posible.

Divide las casas en solo dos categorías. El precio pronosticado para cualquier casa en consideración es el precio promedio histórico de las casas en la misma categoría.

Utilizamos datos para decidir cómo dividir las casas en dos grupos, y luego nuevamente para determinar el precio pronosticado en cada grupo. Este paso de capturar patrones a partir de los datos se denomina ajuste o entrenamiento del modelo. Los datos utilizados para ajustar el modelo se llaman datos de entrenamiento.

Los detalles de cómo se ajusta el modelo (por ejemplo, cómo dividir los datos) son lo suficientemente complejos como para dejarlos para más adelante. Después de que el modelo se haya ajustado, puedes aplicarlo a nuevos datos para predecir los precios de hogares adicionales.

### Mejorando el Árbol de Decisión
¿Cuál de los dos árboles de decisión siguientes es más probable que resulte del ajuste de los datos de entrenamiento de bienes raíces?

Primeros Árboles de Decisión

El árbol de decisión de la izquierda (Árbol de Decisión 1) probablemente tiene más sentido, porque captura la realidad de que las casas con más habitaciones tienden a venderse a precios más altos que las casas con menos habitaciones. La mayor limitación de este modelo es que no captura la mayoría de los factores que afectan el precio de la vivienda, como el número de baños, el tamaño del lote, la ubicación, etc.

Puedes capturar más factores utilizando un árbol que tenga más "divisiones". A estos se les llama árboles "más profundos". Un árbol de decisión que también considere el tamaño total del lote de cada casa podría verse así: Árbol de Profundidad 2

Predices el precio de cualquier casa siguiendo el árbol de decisión, siempre eligiendo el camino correspondiente a las características de esa casa. El precio pronosticado para la casa se encuentra en la parte inferior del árbol. El punto en la parte inferior donde realizamos una predicción se llama hoja.

Las divisiones y los valores en las hojas se determinarán mediante los datos, así que es hora de que revises los datos con los que trabajarás.

### Usando Pandas para familiarizarte con tus datos
El primer paso en cualquier proyecto de aprendizaje automático es familiarizarse con los datos. Utilizarás la biblioteca Pandas para esto. Pandas es la principal herramienta que utilizan los científicos de datos para explorar y manipular datos. La mayoría de las personas abrevian pandas en su código como pd. Hacemos esto con el comando

In [1]:
import pandas as pd

### Seleccionando datos para el modelado
Tu conjunto de datos tiene demasiadas variables como para entenderlo completamente, e incluso para imprimirlo de manera ordenada. ¿Cómo puedes reducir esta abrumadora cantidad de datos a algo que puedas entender?

Comenzaremos seleccionando algunas variables utilizando nuestra intuición. En cursos posteriores, se te mostrarán técnicas estadísticas para priorizar automáticamente las variables.

Para elegir variables/columnas, necesitaremos ver una lista de todas las columnas en el conjunto de datos. Esto se hace con la propiedad "columns" del DataFrame (la última línea de código a continuación).

In [2]:
import pandas as pd

melbourn_data = pd.read_csv("melb_data.csv")
melbourn_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [3]:
melbourn_data = melbourn_data.dropna(axis=0)

Por convención, el objetivo de predicción se llama y. Entonces, el código que necesitamos para guardar los precios de las casas en los datos de Melbourne es el siguiente:

In [4]:
y = melbourn_data.Price

In [5]:
caracteristicas = ["Rooms","Bathroom","Landsize","Lattitude","Longtitude"]

In [6]:
X = melbourn_data[caracteristicas]

In [7]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [8]:
X.head(2)

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944


### Construyendo tu modelo
Utilizarás la biblioteca scikit-learn para crear tus modelos. Cuando escribas el código, esta biblioteca se llama sklearn, como verás en el código de ejemplo. Scikit-learn es fácilmente la biblioteca más popular para modelar los tipos de datos que se suelen almacenar en DataFrames.

Los pasos para construir y utilizar un modelo son los siguientes:

* Definir: ¿Qué tipo de modelo será? ¿Un árbol de decisión? ¿Algún otro tipo de modelo? También se especifican otros parámetros del tipo de modelo.
* Ajustar (Fit): Capturar patrones a partir de los datos proporcionados. Este es el corazón del modelado.
* Predecir: Exactamente lo que suena, hacer predicciones.
* Evaluar: Determinar qué tan precisas son las predicciones del modelo.
Aquí tienes un ejemplo de cómo definir un modelo de árbol de decisión con scikit-learn y ajustarlo con las características y la variable objetivo.

In [9]:
from sklearn.tree import DecisionTreeClassifier

melbourn_model = DecisionTreeClassifier(random_state=1)

# entrenando el modelo

melbourn_model.fit(X, y)

DecisionTreeClassifier(random_state=1)

In [10]:
print("Haciendo predicciones para las siguientes 5 casas")
print(X.head())
print("Las predicciones")
print(melbourn_model.predict(X.head()))

Haciendo predicciones para las siguientes 5 casas
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
4      4       1.0     120.0   -37.8072    144.9941
6      3       2.0     245.0   -37.8024    144.9993
7      2       1.0     256.0   -37.8060    144.9954
Las predicciones
[1035000. 1465000. 1600000. 1876000. 1636000.]


Has construido un modelo. Pero, ¿qué tan bueno es?

En esta lección, aprenderás a utilizar la validación del modelo para medir la calidad de tu modelo. Medir la calidad del modelo es clave para mejorar iterativamente tus modelos.

¿Qué es la validación del modelo?
Querrás evaluar casi todos los modelos que construyas. En la mayoría (aunque no en todos) de los casos, la medida relevante de la calidad del modelo es la precisión predictiva. En otras palabras, ¿las predicciones del modelo serán cercanas a lo que realmente sucede?

Muchas personas cometen un gran error al medir la precisión predictiva. Realizan predicciones con sus datos de entrenamiento y comparan esas predicciones con los valores objetivo en los datos de entrenamiento. Veremos el problema con este enfoque y cómo resolverlo en un momento, pero primero pensemos cómo haríamos esto.

Primero, tendrías que resumir la calidad del modelo de una manera comprensible. Si comparas los valores predichos y reales de las viviendas para 10,000 casas, es probable que encuentres una mezcla de predicciones buenas y malas. Revisar una lista de 10,000 valores predichos y reales sería inútil. Necesitamos resumir esto en una única métrica.

Existen muchas métricas para resumir la calidad del modelo, pero comenzaremos con una llamada Error Absoluto Medio (también conocido como MAE, por sus siglas en inglés). Desglosemos esta métrica comenzando por la última palabra, error.

El error de predicción para cada casa es:

error = real - predicho
Entonces, si una casa cuesta $150,000 y predijiste que costaría $100,000, el error es de $50,000.

Con la métrica MAE, tomamos el valor absoluto de cada error. Esto convierte cada error en un número positivo. Luego, calculamos el promedio de esos errores absolutos. Esta es nuestra medida de calidad del modelo. En términos sencillos, se puede decir:

En promedio, nuestras predicciones están equivocadas por aproximadamente X.

Para calcular el MAE, primero necesitamos un modelo. Ese modelo está construido en una celda oculta a continuación, que puedes revisar haciendo clic en el botón de código.

In [11]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourn_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

1149.0477727566172

El problema con las puntuaciones "en la muestra"
La medida que acabamos de calcular se puede llamar una puntuación "en la muestra". Utilizamos una única "muestra" de viviendas tanto para construir el modelo como para evaluarlo. Aquí está el problema con esto.

Imagina que, en el gran mercado inmobiliario, el color de la puerta no tiene relación con el precio de la vivienda.

Sin embargo, en la muestra de datos que utilizaste para construir el modelo, todas las viviendas con puertas verdes eran muy caras. El trabajo del modelo es encontrar patrones que predigan los precios de las viviendas, por lo que detectará este patrón y siempre pronosticará precios altos para las viviendas con puertas verdes.

Dado que este patrón se derivó de los datos de entrenamiento, el modelo parecerá preciso en los datos de entrenamiento.

Pero si este patrón no se cumple cuando el modelo ve nuevos datos, el modelo sería muy impreciso cuando se utiliza en la práctica.

Dado que el valor práctico de los modelos radica en hacer predicciones sobre nuevos datos, medimos su rendimiento en datos que no se utilizaron para construir el modelo. La forma más sencilla de hacer esto es excluir algunos datos del proceso de construcción del modelo y luego usar esos datos para evaluar la precisión del modelo en datos que no ha visto antes. A estos datos se les llama datos de validación.

El problema con las puntuaciones "en la muestra"
La medida que acabamos de calcular se puede llamar una puntuación "en la muestra". Utilizamos una única "muestra" de viviendas tanto para construir el modelo como para evaluarlo. Aquí está el problema con esto.

Imagina que, en el gran mercado inmobiliario, el color de la puerta no tiene relación con el precio de la vivienda.

Sin embargo, en la muestra de datos que utilizaste para construir el modelo, todas las viviendas con puertas verdes eran muy caras. El trabajo del modelo es encontrar patrones que predigan los precios de las viviendas, por lo que detectará este patrón y siempre pronosticará precios altos para las viviendas con puertas verdes.

Dado que este patrón se derivó de los datos de entrenamiento, el modelo parecerá preciso en los datos de entrenamiento.

Pero si este patrón no se cumple cuando el modelo ve nuevos datos, el modelo sería muy impreciso cuando se utiliza en la práctica.

Dado que el valor práctico de los modelos radica en hacer predicciones sobre nuevos datos, medimos su rendimiento en datos que no se utilizaron para construir el modelo. La forma más sencilla de hacer esto es excluir algunos datos del proceso de construcción del modelo y luego usar esos datos para evaluar la precisión del modelo en datos que no ha visto antes. A estos datos se les llama datos de validación.

In [12]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

melbourne_model = DecisionTreeClassifier()

# entrenando el modelo

melbourne_model.fit(train_X, train_y)

val_prediccion = melbourne_model.predict(test_X)
print(mean_absolute_error(test_y, val_prediccion))

289567.58225806453


¡Wow!

El error absoluto medio para los datos dentro de la muestra fue de aproximadamente 1100 dólares. Sin embargo, para los datos fuera de la muestra, es de más de 360,000 dólares.

Esta diferencia representa la brecha entre un modelo que es casi exactamente correcto y uno que no es utilizable para la mayoría de los propósitos prácticos. Como punto de referencia, el valor promedio de las viviendas en los datos de validación es de 1.1 millones de dólares. Por lo tanto, el error en los nuevos datos es aproximadamente una cuarta parte del valor promedio de una vivienda.

Existen muchas formas de mejorar este modelo, como experimentar para encontrar mejores características o diferentes tipos de modelos.

Al final de este paso, comprenderás los conceptos de subajuste (underfitting) y sobreajuste (overfitting), y podrás aplicar estas ideas para hacer que tus modelos sean más precisos.

Experimentando con diferentes modelos
Ahora que tienes una forma confiable de medir la precisión del modelo, puedes experimentar con modelos alternativos y ver cuál proporciona las mejores predicciones. Pero ¿qué alternativas tienes para los modelos?

Puedes ver en la documentación de scikit-learn que el modelo de árbol de decisión tiene muchas opciones (más de las que necesitarás por un largo tiempo). Las opciones más importantes determinan la profundidad del árbol. Recuerda de la primera lección de este curso que la profundidad de un árbol es una medida de cuántas divisiones realiza antes de llegar a una predicción. Este es un árbol relativamente poco profundo.

En la práctica, no es raro que un árbol tenga 10 divisiones entre el nivel superior (todas las casas) y una hoja. A medida que el árbol se vuelve más profundo, el conjunto de datos se divide en hojas con menos casas. Si un árbol solo tuviera 1 división, dividiría los datos en 2 grupos. Si cada grupo se divide nuevamente, tendríamos 4 grupos de casas. Dividir cada uno de esos grupos nuevamente crearía 8 grupos. Si seguimos duplicando el número de grupos agregando más divisiones en cada nivel, tendremos 210 grupos de casas para cuando lleguemos al décimo nivel. Eso son 1024 hojas.

Cuando dividimos las casas en muchas hojas, también tenemos menos casas en cada hoja. Las hojas con muy pocas casas harán predicciones que se acercan mucho a los valores reales de esas casas, pero pueden hacer predicciones muy poco confiables para nuevos datos (porque cada predicción se basa en solo unas pocas casas).

Esto es un fenómeno llamado sobreajuste (overfitting), donde un modelo se ajusta casi perfectamente a los datos de entrenamiento, pero tiene un mal desempeño en la validación y otros nuevos datos. Por otro lado, si hacemos nuestro árbol muy poco profundo, no divide las casas en grupos muy distintos.

En el extremo, si un árbol divide las casas en solo 2 o 4 grupos, cada grupo seguirá teniendo una amplia variedad de casas. Las predicciones resultantes pueden estar muy alejadas para la mayoría de las casas, incluso en los datos de entrenamiento (y también serán malas en la validación por la misma razón). Cuando un modelo no logra capturar distinciones y patrones importantes en los datos, por lo que tiene un mal rendimiento incluso en los datos de entrenamiento, eso se llama subajuste (underfitting).

Dado que nos importa la precisión en nuevos datos, que estimamos a partir de nuestros datos de validación, queremos encontrar el punto óptimo entre el subajuste y el sobreajuste. Visualmente, queremos el punto más bajo de la curva de validación (roja) en la figura a continuación.

Ejemplo
Existen algunas alternativas para controlar la profundidad del árbol, y muchas permiten que algunas rutas a través del árbol tengan mayor profundidad que otras rutas. Pero el argumento **max_leaf_nodes** proporciona una forma muy sensata de controlar el sobreajuste frente al subajuste. Cuantas más hojas permitamos que haga el modelo, más nos moveremos desde el área de subajuste en el gráfico anterior hacia el área de sobreajuste.

Podemos usar una función de utilidad para ayudar a comparar las puntuaciones de error absoluto medio (MAE) para diferentes valores de max_leaf_nodes:

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_nodos, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_nodos, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [14]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    # crea el modelo
    model.fit(train_X, train_y)
    # entreno el modelo
    preds_val = model.predict(val_X)
    # predice
    mae = mean_absolute_error(val_y, preds_val) #retorna
    return(mae)

# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  385881
Max leaf nodes: 50  		 Mean Absolute Error:  280417
Max leaf nodes: 500  		 Mean Absolute Error:  262176
Max leaf nodes: 5000  		 Mean Absolute Error:  278532


De las opciones enumeradas, 500 es el número óptimo de hojas.

Conclusión
Aquí está el resumen: Los modelos pueden sufrir de:

**Sobreajuste:** capturan patrones espurios que no se repetirán en el futuro, lo que lleva a predicciones menos precisas.
**Subajuste:** no capturan patrones relevantes, lo que también lleva a predicciones menos precisas.
Utilizamos datos de validación, que no se utilizan en el entrenamiento del modelo, para medir la precisión de un modelo candidato. Esto nos permite probar muchos modelos candidatos y quedarnos con el mejor.

Los árboles de decisión te enfrentan a una decisión difícil. Un árbol profundo con muchas hojas sobreajustará porque cada predicción proviene de datos históricos de solo unas pocas casas en su hoja. Pero un árbol superficial con pocas hojas tendrá un rendimiento deficiente porque no captura tantas distinciones en los datos sin procesar.

Incluso las técnicas de modelado más sofisticadas de hoy en día enfrentan esta tensión entre el subajuste y el sobreajuste. Sin embargo, muchos modelos tienen ideas ingeniosas que pueden llevar a un mejor rendimiento. Vamos a analizar el bosque aleatorio como ejemplo. (RANDOM FOREST)

El bosque aleatorio utiliza muchos árboles y realiza una predicción promediando las predicciones de cada árbol componente. Generalmente, tiene una precisión predictiva mucho mejor que un solo árbol de decisión y funciona bien con los parámetros predeterminados. Si sigues modelando, puedes aprender más modelos con un rendimiento aún mejor, pero muchos de ellos son sensibles a obtener los parámetros correctos.

Has visto el código para cargar los datos varias veces. Al finalizar la carga de datos, tenemos las siguientes variables:

train_X
val_X
train_y
val_y

Construimos un modelo de bosque aleatorio de manera similar a cómo construimos un árbol de decisión en scikit-learn, pero esta vez utilizando la clase RandomForestRegressor en lugar de DecisionTreeRegressor.

In [20]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(test_X)
print(mean_absolute_error(test_y, melb_preds))

202743.18372721455
